In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib
matplotlib.pyplot.style.use('seaborn')
matplotlib.rcParams['figure.figsize'] = (15, 5)

%matplotlib inline

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
np.set_printoptions(precision=2, suppress=True)

In [ ]:
import math
import copy

import scipy.stats as stats

In [ ]:
from sklearn import model_selection, metrics, datasets

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Model

# $Model(x_i, w) = w_0 + \sum_j^d w_j x_i^j$

$x_i$ - $i'th$ featrue in matrix of features $X$

$x_i^j$ - $j'th$ featrue value of current feature $i$

$w_j$ - weight for $x_i^j$ feature value

$w_0$ - free coeficent

---

In matrix (add $x_i^0$ and set it to $1$, and add $w_0$ for it)

$Model(X, w) = Xw$

$\begin{bmatrix}
(x_0^0=1), & ..., & (x_0^j) \\
(x_1^0=1), & ..., & (x_1^j) \\
... \\
(x_n^0=1), & ..., & (x_n^j) \\
\end{bmatrix}$
$\begin{bmatrix}
w_0 \\
...\\
w_j
\end{bmatrix}$

In [ ]:
X = np.array([
    [1, 2, 5],
    [1, 5, 5],
    [1, 8, 5],
], dtype=np.float64)

w = np.array([0.1, 0.1, 0.1], dtype=np.float64)

yhat = X.dot(w)
yhat

##### $y = Xw + \epsilon$

$\begin{bmatrix}
y_0 \\
... \\
y_n \\
\end{bmatrix} = $
$\begin{bmatrix}
x_0^0 & x_0^1 &  ... & x_0^j \\
... \\
x_n^0 & x_n^1 &  ... & x_n^j
\end{bmatrix}$
$\begin{bmatrix}
w_0 \\
...\\
w_j
\end{bmatrix} +$
$\begin{bmatrix}
\epsilon_0 \\
...\\
\epsilon_n
\end{bmatrix}$

# Classification

### Link function: squeeze regression real line into $[0,1]$ (and make it probability)

$P(y=1|X, w) = Sigmoid(Model(X, w))$

---

### Logistic function (sigmoid, logit)

### $Sigmoid(X, w) = \frac{1}{1 + e^{-Model(X,w)}}$

$Sigmoid(X, w) = \frac{1}{1 + e^{-(w_0 + \sum_j^d w_j x_i^j)}}$

In [ ]:
def sigmoid(x, w=1):
    # w - coeff weights is a constant to a function, makin' function tighter, also makes confidence borders lower
    return 1 / (1 + np.exp(-x*w))

In [ ]:
xs = np.linspace(-5, 5, 100)
ys = [sigmoid(x) for x in xs]
plt.plot(xs, ys);

In [ ]:
X = np.array([
    [1, 2, 5],
    [1, 5, 5],
    [1, 8, 5],
], dtype=np.float64)

w = np.array([0.1, 0.1, 0.1], dtype=np.float64)

proba = sigmoid(X.dot(w))
proba

# Optimization task

### maximum likelihood estimation (MLE) 

### $\ell(w) = \prod_i^N P(y_i | x_i, w)$

### $\ell\ell(w) = \ln\prod_i^N P(y_i | x_i, w)$

### $\ell\ell(w) \rightarrow \underset{w}{max}$

### Compute derivative of log likelihood with respect to a single coefficient

$\frac{\partial\ell}{\partial w_j} = \sum_{i=1}^N\Big(
x_i \cdot \left([y_i == 1] - P(y_i = 1 | \mathbf{x}_i, \mathbf{w})\right)
\Big)$

The log likelihood: 

$\frac{\partial}{\partial w_j} \ell\ell(\mathbf{w}) = \sum_{i=1}^N \Big( ([y_i == 1] - 1)\mathbf{w}^T x_i - \ln\left(1 + e^{-\mathbf{w}^T x_i}\right) \Big) $

##### * * * derivation

$\ell(w) = \prod_i^N P(y_i | x_i, w)$

log turns into sumation and does not change maximum optima:

$\ell\ell(w) = \ln\prod_i^N P(y_i | x_i, w)$

$\ln\prod_i^N P(y_i | x_i, w) \rightarrow \sum_i^N \ln P(y_i | x_i, w)$

---

$\sum_i^N \ln P(y_i | x_i, w)$

$\sum_i^N\left(
[y_i == 1] \ln P(y_i = 1 | x_i, w) +
[y_i == 0] \ln P(y_i = 0 | x_i, w)
\right)$

$P(y_i = 1 | x_i, w) = \frac{1}{1 + e^{-w^Tx}}$

$P(y_i = 0 | x_i, w) = \frac{e^{-w^Tx}}{1 + e^{-w^Tx}}$

$\sum_i^N\left(
[y_i == 1] \ln \frac{1}{1 + e^{-w^Tx}} +
[y_i == 0] \ln  \frac{e^{-w^Tx}}{1 + e^{-w^Tx}}
\right)$

$\sum_{i=1}^N \Big( -(1 - [y_i == 1])\mathbf{w}^T x_i - \ln\left(1 + e^{-\mathbf{w}^T x_i}\right) \Big)$

$\sum_{i=1}^N \Big( ([y_i == 1] - 1)\mathbf{w}^T x_i - \ln\left(1 + e^{-\mathbf{w}^T x_i}\right) \Big)$

# Gradient Ascent (MLE)

$w^t = w^{t-1} + \alpha_t \triangledown \ell\ell(w^{t-1},X,y) - \lambda_t \left\| w \right\|_2^2$

$\triangledown$ - gradient _(vector of derivatives)_

$\alpha_t$ - learning rate for current step

$\lambda \left\| w \right\|_2^2$ - l2 regularization _(square of second norm)_

$t$ - iteration

Stop: $\left\| w_t - w^{t-1} \right\| < \epsilon$

# TODO:

In [ ]:
def sigmoid(w, X):
    return 1 / (1 + np.exp(-w.dot(X.T)))

In [ ]:
def mle(w, X):
    return np.log(np.prod(sigmoid(w, X)))

In [ ]:
def gmle(w, X, y):
    indicator = y - 1 # $[y_i == 1] - 1$
    return (indicator * w.dot(X.T)) - np.log(1 + np.exp(-w.dot(X.T)))

In [ ]:
def ridge(w, l):
    w = w.copy()
    w[0] = 0 # Don’t penalize intercept term w0
    return 2 * l * w

In [ ]:
def maximize(X, y, cost, grad, reg):
    # add coef of ones
    X = np.append(np.ones(len(X[0])), X.T).reshape(4,3).T

    # weights vector
    w = np.zeros(len(X[0]), dtype=np.float64)

    # parameters
    epsilon = 0.0
    alpha = 0.1
    weights = [w]
    error = []
    
    for iteration in range(500):
        print(w)
        print(grad(w, X, y))
        w = w - alpha * grad(w, X, y) - reg(w, 0.00001)
        if np.linalg.norm(w - weights[-1]) < epsilon:
            break
        weights.append(w)
        error.append(cost(w, X))

    return w[0], w[1:], error

In [ ]:
def predict(X, w, coef):
    X = np.append(np.ones(len(X[0])), X.T).reshape(4,3).T
    w = np.append(np.array(coef), w)

    return X.dot(w)

In [ ]:
X = np.array([
    [0.2, 0.15, 0.8],
    [0.5, 0.45, 0.12],
    [0.8, 0.53, 0.33],
], dtype=np.float64)

y = np.array([
    0,
    1,
    0,
], dtype=np.int)

In [ ]:
coef, w, error = maximize(X, y, mle, gmle, ridge)

coef

w
predict(X, w, coef)

In [ ]:
plt.plot(error);

# Check by sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression().fit(X, y)

reg.intercept_ 

reg.coef_
reg.predict(X)